In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/red-table/Red_Table.xlsx


# 과제 1. 매장 소개 문구 자동 출력 코드 구현
- 과제는 캐글에서 진행했습니다.
- https://www.kaggle.com/kkj214/red-table-1

In [2]:
!pip install openpyxl
#from google.colab import drive
#import pandas as pd
#import numpy as np
#!pip install openpyxl
#import re 
#path = "/content/sample_data/Red_Table.xlsx"

     |████████████████████████████████| 242 kB 894 kB/s            


# 과제 1. 매장 소개 문구 자동 출력 코드 구현
- 과제는 캐글에서 진행했습니다.
- https://www.kaggle.com/kkj214/red-table-1

## 데이터 확인

In [3]:
sd = pd.read_excel("../input/red-table/Red_Table.xlsx",sheet_name = '1번데이터')#, index_col = 0)
# sd = Store Data / 1번데이터

se = pd.read_excel("../input/red-table/Red_Table.xlsx",sheet_name = '1번출력문장', index_col = 0)
# sd = Store Data / 1번 출력문장

In [4]:
#1번 데이터의 기본 정보 확인
sd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   매장명      100 non-null    object 
 1   지점명      9 non-null      object 
 2   도로명주소    100 non-null    object 
 3   지번주소     100 non-null    object 
 4   어워드      93 non-null     object 
 5   어워드수여년도  74 non-null     float64
 6   어워드등급    27 non-null     object 
 7   반려동물출입   100 non-null    object 
 8   남녀화장실구분  100 non-null    object 
 9   배리어프리    100 non-null    object 
dtypes: float64(1), object(9)
memory usage: 7.9+ KB


In [5]:
# null값 확인
sd.isnull().sum()
# 지점명과 어워드(+년도,등급)에 결측값 존재

매장명         0
지점명        91
도로명주소       0
지번주소        0
어워드         7
어워드수여년도    26
어워드등급      73
반려동물출입      0
남녀화장실구분     0
배리어프리       0
dtype: int64

# 데이터 변경
- '시_도','시_군구','지점명','통합주소' 컬럼을 추가
- 결측값을 제거하였습니다


In [6]:
# 시/도 와 시/군/구 컬럼 추가
sd['시_도'] = sd.도로명주소.apply(lambda x : x.split()[0])
sd['시_군구'] = sd.도로명주소.apply(lambda x : x.split()[1])
#sd['시도_시군구'] = sd.도로명주소.apply(lambda x : [x.split()[0]] + [x.split()[1]])

#지점명 괄호 추가
sd.지점명 = sd.지점명.apply(lambda x : "" if type(x)==float else "("+x+")")
sd['통합주소'] = sd.도로명주소 + " " + sd.매장명 + sd.지점명

In [7]:
# 임시로 null 값 을 공백 혹은 0으로 처리했습니다

sd['어워드'] = sd['어워드'].fillna("")
sd['어워드수여년도'] = sd['어워드수여년도'].fillna(0).astype(int)
sd['어워드등급'] = sd['어워드등급'].fillna("")

- 이번과제에서는 딱히 int type으로 변환할 필요가 없다고 판단해 그냥 두었습니다

In [8]:
# Y 이면 1 / N 이면 0 으로 변환

#sd['반려동물출입'] = sd['반려동물출입'].apply(lambda x : 1 if x == "Y" else 0)
#sd['남녀화장실구분'] = sd['남녀화장실구분'].apply(lambda x : 1 if x == "Y" else 0)
#sd['배리어프리'] = sd['배리어프리'].apply(lambda x : 1 if x == "Y" else 0)

# 출력 문장 데이터 확인 및 리스트로 변환
- 기본문장 / 어워드 정보 / 옵션 정보 
- 3가지로 구분해 진행하였습니다

In [9]:
import random
# 문장 무작위 선택을 위해 random

In [10]:
# 기본 문구 리스트 생성
basic = se.iloc[:9]['출력 문구'].to_list()
basic

['어디 가야 할지 고민이라고요? 그럼 [시도] [시군구] 맛집, "매장명"을/를 추천합니다!',
 '무엇을 먹을지 고민되신다고요? [시도] [시군구]에 계시다면 "매장명"을/를 추천합니다!',
 '"매장명"은/는 [시도] [시군구]에 있습니다.',
 '"매장명"을/를 [시도] [시군구]의 가 볼 만한 식당으로 추천합니다!',
 '[시도] [시군구]에서 맛집을 찾으신다면 "매장명"을/를 추천합니다.',
 '[시도] [시군구]에 가신다면 "매장명"에 방문해보세요~',
 '[시도] [시군구]에서 가 볼 만한 식당을 찾으신다면? "매장명"을/를 추천합니다!',
 '[시도] [시군구]에서 식당을 찾으신다면? "매장명"을/를 방문해보세요!',
 '[시도] [시군구]에서 어디를 갈지 고민이라면! "매장명"에 가보시는 건 어떨까요?']

- 코드를 좀 더 효율적으로 짜야할 필요가 있을 것 같습니다

In [11]:
# 랜덤으로 기본 문구에 시도, 시군구, 매장명 추가한 컬럼 생성
sd['기본문장'] = sd['통합주소'].apply(lambda x : random.choice(basic).replace('[시도]',x.split()[0]).replace('[시군구]',x.split()[1]).replace('매장명',x.split()[-1]))
sd['기본문장']                             

0        부산광역시 강서구에서 어디를 갈지 고민이라면! "명지횟집"에 가보시는 건 어떨까요?
1     어디 가야 할지 고민이라고요? 그럼 서울특별시 종로구 맛집, "동성각"을/를 추천합니다!
2       서울특별시 용산구에서 어디를 갈지 고민이라면! "후렌드치킨"에 가보시는 건 어떨까요?
3     어디 가야 할지 고민이라고요? 그럼 서울특별시 강남구 맛집, "브래서리"을/를 추천...
4     어디 가야 할지 고민이라고요? 그럼 서울특별시 영등포구 맛집, "린네스가든"을/를 ...
                            ...                        
95                          "나루터연화"은/는 부산광역시 기장군에 있습니다.
96      부산광역시 기장군에서 어디를 갈지 고민이라면! "나루터연화"에 가보시는 건 어떨까요?
97           "1960청원모밀"을/를 광주광역시 동구의 가 볼 만한 식당으로 추천합니다!
98                광주광역시 동구에서 식당을 찾으신다면? "청하"을/를 방문해보세요!
99                             "태화장"은/는 대전광역시 동구에 있습니다.
Name: 기본문장, Length: 100, dtype: object

- 은/는 혹은 이/가 등의 경우 ord,chr을 통해 아스키코드로 함수화하여 구별했습니다

In [12]:
# 을/를 받침 확인 함수
def e_w(kstr): #ends with
    k = kstr[-1]
    if "가" <= k <= "힣":
        return (ord(k)-ord("가")) % 28 > 0
    else:
        return

In [13]:
# 은/는, 을/를 구분 -> 비효율적인 코드인것 같아서 변경이 필요해보임 -> (지점명) 괄호의 경우
bs = sd['기본문장'].tolist()
for i in bs :
    if "을/를"  in i : # 을/를의 경우
        if i[i.index("을/를")-2] == ")" :
            if e_w(i[i.index("을/를")-3]) == True :
                bs[bs.index(i)] = i.replace('을/를','을')
            else :
                bs[bs.index(i)] = i.replace('을/를','를')
        else :
            if e_w(i[i.index("을/를")-2]) == True:
                bs[bs.index(i)] = i.replace('을/를','을')
            else:  
                bs[bs.index(i)] = i.replace('을/를','를')
    elif "은/는" in i:
        if i[i.index("은/는")-2] == ")" :
            if e_w(i[i.index("은/는")-3]) == True :
                bs[bs.index(i)] = i.replace('은/는','은')
            else :
                bs[bs.index(i)] = i.replace('은/는','는')
        else :
            if e_w(i[i.index("은/는")-2]) == True:
                bs[bs.index(i)] = i.replace('은/는','은')
            else:  
                bs[bs.index(i)] = i.replace('은/는','는')
sd['기본문장'] = bs

## 수상 문장 작업

In [14]:
# 수상 정보 리스트 생성
award_list = se.iloc[9:21]['출력 문구'].to_list()
award_list

['0000년 지방자치단체 인증을 받은 농림축산식품부 제공 안심식당입니다.',
 '위생 수준이 우수하고 친절한 서비스로 0000년 지방자치단체의 선정을 받은 모범음식점입니다.',
 "0000년 식품의약품안전처 음식점 위생등급 평가를 통해 '[grade]' 등급을 지정받은 곳입니다.",
 "안전하고 위생적인 환경을 갖추었으며, 영양상 균형 잡힌 식품을 조리해 판매하는 '어린이 우수판매업소'입니다.",
 "이 매장은 블루리본 서베이에서 리본 1개를 받은 식당으로, '시간을 내어 다시 방문하고 싶은 곳'이라 평가받았습니다.",
 "이 매장은 블루리본 서베이에서 리본 2개를 받아, '주위 사람들에게도 추천하고 싶은 곳'이라는 좋은 평가를 받았습니다.",
 "이 매장은 블루리본 서베이에서 리본 3개를 받은 매장으로, '자신의 분야에서 가장 뛰어난 솜씨를 보이는 곳'이라는 훌륭한 평가를 받았습니다.",
 '미쉐린 가이드 서울 2021에서 1스타를 획득했습니다. 훌륭한 요리를 맛볼 수 있는 레스토랑입니다.',
 '미쉐린 가이드 서울 2021에서 2스타를 획득했습니다. 훌륭한 요리를 맛볼 수 있어 멀리 찾아갈 만한 가치가 있는 레스토랑입니다.',
 '미쉐린 가이드 서울 2021에서 3스타를 획득했습니다. 매우 훌륭한 요리를 맛볼 수 있어 특별한 여행을 떠날 가치가 있는 레스토랑입니다.',
 '미쉐린 가이드 서울 2021의 미쉐린 플레이트로 선정된 레스토랑입니다. 신선한 재료로 만든 좋은 요리를 맛볼 수 있습니다.',
 '미쉐린 가이드 서울 2021의 빕 구르망으로 선정된 매장입니다. 합리적인 가격에 훌륭한 음식을 맛볼 수 있습니다.']

In [15]:
sd['어워드수여년도'] = sd['어워드수여년도'].astype(str)
sd['어워드통합'] = sd[['어워드','어워드수여년도','어워드등급']].apply(" ".join, axis=1)

- 미슐랭의 경우 빕구르망, 블루리본의 경우 1,2만 존재했기때문에 나머지 등급은 제외하였습니다

In [16]:
sd['어워드정보'] = sd['어워드통합'].apply(lambda x : award_list[0].replace("0000",x.split()[1]) if x.split()[0] == '안심식당' 
               else (award_list[1].replace("0000",x.split()[1]) if x.split()[0] == '모범음식점' 
                     else (award_list[2].replace("0000",x.split()[1])) if x.split()[0] == '위생등급'
                     else (award_list[4] if x.split()[-1] == 'RIBBON_ONE'
                     else (award_list[5] if x.split()[-1] == 'RIBBON_TWO'
                     else (award_list[-1] if x.split()[-1] == '구르망'
                     else ""
                    )
                ))))

# 옵션 정보 전처리

In [17]:
sd['옵션통합'] = sd[['반려동물출입','남녀화장실구분','배리어프리']].apply(" ".join, axis=1).apply(lambda x : x.split())

In [18]:
#옵션 리스트 생성
option_list = se.iloc[21:]['출력 문구'].to_list()
ol = option_list.copy()
option_list

['반려동물 입장이 가능한 매장입니다.',
 '남녀 화장실이 구분되어 있는 매장입니다.',
 '베리어프리 매장입니다.',
 '반려동물 입장이 가능하고, 남녀 화장실이 구분되어 있는 매장입니다.',
 '반려동물 입장이 가능하며 베리어프리 매장입니다.',
 '남녀 화장실이 구분되어 있으며 베리어프리 매장입니다.',
 '반려동물 입장이 가능하고, 남녀 화장실이 구분되어 있으며 베리어프리 매장입니다.']

- 반려동물출입과 배리어프리가 가능한 곳은 한곳도 없었습니다
- 남녀화장실구분만 18곳 가능했음

In [19]:
for i in ['반려동물출입','남녀화장실구분','배리어프리'] :
    print(sd[i].value_counts(),'\n')

N    100
Name: 반려동물출입, dtype: int64 

N    82
Y    18
Name: 남녀화장실구분, dtype: int64 

N    100
Name: 배리어프리, dtype: int64 



In [20]:
sd['옵션정보'] = sd['옵션통합'].apply(lambda x : option_list[1] if x[1] == 'Y' else "")

In [21]:
sd['소개문구'] = sd[['기본문장','어워드정보','옵션정보']].apply(" ".join,axis = 1 )

In [22]:
sd[['매장명','소개문구']].to_excel('레드테이블_과제_1_김경준.xlsx', index=False)
sd[['매장명','소개문구']]

,매장명,소개문구
0,명지횟집,"부산광역시 강서구에서 어디를 갈지 고민이라면! ""명지횟집""에 가보시는 건 어떨까요?..."
1,동성각,"어디 가야 할지 고민이라고요? 그럼 서울특별시 종로구 맛집, ""동성각""을 추천합니다..."
2,후렌드치킨,"서울특별시 용산구에서 어디를 갈지 고민이라면! ""후렌드치킨""에 가보시는 건 어떨까요?"
3,브래서리,"어디 가야 할지 고민이라고요? 그럼 서울특별시 강남구 맛집, ""브래서리""를 추천합니..."
4,카페 린네스가든,"어디 가야 할지 고민이라고요? 그럼 서울특별시 영등포구 맛집, ""린네스가든""을 추천..."
...,...,...
95,나루터연화,"""나루터연화""는 부산광역시 기장군에 있습니다. 위생 수준이 우수하고 친절한 서비스로..."
96,나루터연화,"부산광역시 기장군에서 어디를 갈지 고민이라면! ""나루터연화""에 가보시는 건 어떨까요..."
97,1960청원모밀,"""1960청원모밀""을 광주광역시 동구의 가 볼 만한 식당으로 추천합니다! 이 매장은..."
98,청하,"광주광역시 동구에서 식당을 찾으신다면? ""청하""를 방문해보세요! 위생 수준이 우수하..."


# 리뷰
- 거의 문자열만을 다룬 데이터였기때문에 그래도 소개문구까지 생성할 수 있었던 것 같습니다.
- 하지만 효율적인 코드를 위해 많이 공부해야할 필요성을 느꼈습니다